# Part III: Core FastAPI Concepts

## Chapter 7: Request Handling and Context

While FastAPI's path operation parameters and Pydantic models handle most data extraction automatically, sometimes you need direct access to the HTTP request itself. This chapter explores how to work with the `Request` object, handle form data and file uploads, manage cookies, and manipulate HTTP headers for complete control over request and response handling.

---

### 7.1 The `Request` Object: Direct Access to Headers, Cookies, and Client Info

The `Request` object in FastAPI (from Starlette) provides direct access to the raw HTTP request. This is useful when you need information that isn't automatically extracted by path parameters or when building middleware and advanced features.

#### Accessing the Request Object

To access the request object, import it from `fastapi` and add it as a parameter to your path operation:

```python
from fastapi import FastAPI, Request

app = FastAPI()


@app.get("/info")
async def get_request_info(request: Request):
    """
    Access various properties of the HTTP request.
    """
    return {
        "method": request.method,
        "url": str(request.url),
        "path": request.url.path,
        "query_params": dict(request.query_params),
        "headers": dict(request.headers),
        "client_host": request.client.host if request.client else None,
        "client_port": request.client.port if request.client else None,
    }
```

#### Request Properties

The `Request` object provides numerous properties for accessing request data:

```python
from fastapi import FastAPI, Request

app = FastAPI()


@app.get("/request-details")
async def request_details(request: Request):
    """
    Comprehensive example of request properties.
    """
    return {
        # Basic request info
        "method": request.method,  # HTTP method (GET, POST, etc.)
        "url": str(request.url),  # Full URL as string
        "path": request.url.path,  # Path portion of URL
        "query_string": request.url.query,  # Query string portion
        "scheme": request.url.scheme,  # http or https
        "http_version": request.scope["http_version"],  # HTTP version
        
        # Client information
        "client_host": request.client.host if request.client else None,
        "client_port": request.client.port if request.client else None,
        
        # Headers (case-insensitive access)
        "user_agent": request.headers.get("user-agent"),
        "accept": request.headers.get("accept"),
        "content_type": request.headers.get("content-type"),
        "authorization": request.headers.get("authorization"),
        
        # Query parameters
        "query_params": dict(request.query_params),
        
        # Path parameters (from route)
        "path_params": request.path_params,
        
        # Cookies
        "cookies": request.cookies,
        
        # App and route info
        "app_title": request.app.title,
        "route_name": request.scope.get("route").name if request.scope.get("route") else None,
        
        # Server info
        "server": request.scope.get("server"),
    }
```

#### Working with Headers

Headers are accessible through `request.headers`, which behaves like a case-insensitive dictionary:

```python
from fastapi import FastAPI, Request, HTTPException

app = FastAPI()


@app.get("/headers")
async def get_headers(request: Request):
    """
    Access all headers from the request.
    """
    # Access specific headers (case-insensitive)
    content_type = request.headers.get("content-type")
    user_agent = request.headers.get("user-agent")
    authorization = request.headers.get("authorization")
    
    # Check if header exists
    has_auth = "authorization" in request.headers
    
    # Get all headers as dict
    all_headers = dict(request.headers)
    
    return {
        "content_type": content_type,
        "user_agent": user_agent,
        "has_authorization": has_auth,
        "all_headers": all_headers,
    }


@app.get("/validate-content-type")
async def validate_content_type(request: Request):
    """
    Validate that the request has a specific content type.
    """
    content_type = request.headers.get("content-type")
    
    if not content_type:
        raise HTTPException(status_code=400, detail="Content-Type header is required")
    
    if not content_type.startswith("application/json"):
        raise HTTPException(
            status_code=415,  # Unsupported Media Type
            detail="Content-Type must be application/json",
        )
    
    return {"content_type": content_type}
```

#### Accessing the Request Body

For endpoints that need raw access to the request body:

```python
from fastapi import FastAPI, Request

app = FastAPI()


@app.post("/raw-body")
async def get_raw_body(request: Request):
    """
    Access the raw request body as bytes.
    """
    # Read body as bytes
    body_bytes = await request.body()
    
    # Convert to string
    body_str = body_bytes.decode("utf-8")
    
    return {
        "body_bytes": body_bytes[:100],  # First 100 bytes
        "body_string": body_str[:100],  # First 100 characters
        "body_length": len(body_bytes),
    }


@app.post("/json-body")
async def get_json_body(request: Request):
    """
    Access the request body as JSON.
    This is an alternative to using Pydantic models.
    """
    # Parse body as JSON
    body = await request.json()
    
    return {
        "received_data": body,
        "data_type": str(type(body)),
    }


@app.post("/form-body")
async def get_form_body(request: Request):
    """
    Access the request body as form data.
    """
    # Parse body as form data
    form = await request.form()
    
    return {
        "form_data": dict(form),
    }
```

#### Streaming Request Body

For large files or streaming data, read the body in chunks:

```python
from fastapi import FastAPI, Request

app = FastAPI()


@app.post("/stream-upload")
async def stream_upload(request: Request):
    """
    Stream the request body in chunks.
    Useful for large file uploads.
    """
    chunk_size = 1024 * 1024  # 1MB chunks
    total_size = 0
    chunk_count = 0
    
    # Stream the body
    async for chunk in request.stream():
        chunk_count += 1
        total_size += len(chunk)
        # Process chunk here (save to file, etc.)
        print(f"Received chunk {chunk_count}: {len(chunk)} bytes")
    
    return {
        "total_size": total_size,
        "chunk_count": chunk_count,
    }
```

#### Using Request in Dependencies

The `Request` object can be used in dependencies for advanced scenarios:

```python
from fastapi import FastAPI, Request, Depends, HTTPException
from typing import Annotated

app = FastAPI()


def get_client_info(request: Request):
    """Dependency that extracts client information."""
    return {
        "host": request.client.host if request.client else "unknown",
        "port": request.client.port if request.client else 0,
        "user_agent": request.headers.get("user-agent", "unknown"),
    }


def rate_limiter(request: Request):
    """
    Simple rate limiting dependency.
    In production, use Redis or similar for distributed rate limiting.
    """
    # Simple in-memory rate limiting (not suitable for production)
    client_host = request.client.host if request.client else "unknown"
    
    # Simulated rate limit check
    # In production, check against a store like Redis
    max_requests = 100
    window_seconds = 60
    
    # This is a placeholder - implement actual rate limiting logic
    print(f"Checking rate limit for {client_host}")
    
    return {"client": client_host, "limit": max_requests, "window": window_seconds}


def require_json(request: Request):
    """Dependency that requires JSON content type."""
    content_type = request.headers.get("content-type", "")
    
    if not content_type.startswith("application/json"):
        raise HTTPException(
            status_code=415,
            detail="Content-Type must be application/json",
        )
    
    return True


# Type aliases
ClientInfoDep = Annotated[dict, Depends(get_client_info)]
RateLimitDep = Annotated[dict, Depends(rate_limiter)]


@app.get("/client-info")
async def client_info(client: ClientInfoDep):
    """Get information about the client."""
    return client


@app.post("/limited-endpoint")
async def limited_endpoint(rate: RateLimitDep):
    """Endpoint with rate limiting."""
    return {"message": "Request allowed", "rate_limit_info": rate}
```

#### Request Scope

The `request.scope` dictionary contains low-level information about the request:

```python
from fastapi import FastAPI, Request

app = FastAPI()


@app.get("/scope-info")
async def scope_info(request: Request):
    """
    Access the ASGI scope dictionary.
    Contains low-level request information.
    """
    scope = request.scope
    
    return {
        "type": scope.get("type"),  # "http"
        "asgi_version": scope.get("asgi").get("version") if scope.get("asgi") else None,
        "http_version": scope.get("http_version"),
        "method": scope.get("method"),
        "scheme": scope.get("scheme"),
        "path": scope.get("path"),
        "query_string": scope.get("query_string", b"").decode(),
        "root_path": scope.get("root_path"),
        "headers_count": len(scope.get("headers", [])),
        "server": scope.get("server"),
        "client": scope.get("client"),
        "path_params": scope.get("path_params"),
    }
```

---

### 7.2 Form Data and File Uploads: Handling `multipart/form-data`

Form data is sent as `application/x-www-form-urlencoded` or `multipart/form-data` (for files). FastAPI provides dedicated tools for handling both.

#### Basic Form Data

Use the `Form` class to handle form fields:

```python
from fastapi import FastAPI, Form
from typing import Annotated

app = FastAPI()


@app.post("/login")
async def login(
    username: Annotated[str, Form(min_length=3, max_length=50)],
    password: Annotated[str, Form(min_length=8)],
    remember_me: Annotated[bool, Form()] = False,
):
    """
    Handle a login form submission.
    
    Form fields are extracted from application/x-www-form-urlencoded
    or multipart/form-data request body.
    """
    return {
        "username": username,
        "remember_me": remember_me,
        # Never return passwords!
        "message": "Login successful",
    }
```

**Testing with curl:**

```bash
curl -X POST http://localhost:8000/login \
  -H "Content-Type: application/x-www-form-urlencoded" \
  -d "username=alice&password=secretpassword&remember_me=true"
```

#### Complex Form Data

Handle multiple form fields with validation:

```python
from fastapi import FastAPI, Form, HTTPException
from typing import Annotated
from pydantic import EmailStr
from datetime import date

app = FastAPI()


@app.post("/registration")
async def register(
    first_name: Annotated[str, Form(min_length=1, max_length=100)],
    last_name: Annotated[str, Form(min_length=1, max_length=100)],
    email: Annotated[str, Form()],
    password: Annotated[str, Form(min_length=8, max_length=128)],
    confirm_password: Annotated[str, Form()],
    birth_date: Annotated[str, Form()],
    terms_accepted: Annotated[bool, Form()],
):
    """
    Handle a registration form with validation.
    """
    # Validation
    if password != confirm_password:
        raise HTTPException(status_code=400, detail="Passwords do not match")
    
    if not terms_accepted:
        raise HTTPException(status_code=400, detail="Must accept terms and conditions")
    
    # In production, validate email format, hash password, etc.
    
    return {
        "message": "Registration successful",
        "user": {
            "first_name": first_name,
            "last_name": last_name,
            "email": email,
            "birth_date": birth_date,
        },
    }
```

#### File Uploads with `UploadFile`

The `UploadFile` class provides an async file interface:

```python
from fastapi import FastAPI, UploadFile, File, HTTPException
from typing import Annotated

app = FastAPI()


@app.post("/upload")
async def upload_file(
    file: Annotated[UploadFile, File(description="A file to upload")],
):
    """
    Upload a single file.
    
    UploadFile provides:
    - filename: Original filename
    - content_type: MIME type
    - file: Async file-like object
    - size: File size (after reading)
    """
    # Read file contents
    contents = await file.read()
    
    return {
        "filename": file.filename,
        "content_type": file.content_type,
        "size": len(contents),
    }


@app.post("/upload-info")
async def upload_info(file: UploadFile = File(...)):
    """
    Get file information without reading the entire file.
    """
    return {
        "filename": file.filename,
        "content_type": file.content_type,
        "headers": dict(file.headers),
    }
```

#### Multiple File Uploads

```python
from fastapi import FastAPI, UploadFile, File
from typing import Annotated

app = FastAPI()


@app.post("/upload-multiple")
async def upload_multiple_files(
    files: Annotated[list[UploadFile], File(description="Multiple files to upload")],
):
    """
    Upload multiple files.
    """
    results = []
    
    for file in files:
        contents = await file.read()
        results.append({
            "filename": file.filename,
            "content_type": file.content_type,
            "size": len(contents),
        })
        # Reset file pointer if needed
        await file.seek(0)
    
    return {
        "count": len(results),
        "files": results,
    }
```

#### Combining Form Data and Files

Handle forms with both text fields and files:

```python
from fastapi import FastAPI, Form, File, UploadFile
from typing import Annotated

app = FastAPI()


@app.post("/submit-post")
async def submit_post(
    title: Annotated[str, Form(min_length=1, max_length=200)],
    content: Annotated[str, Form(min_length=1)],
    author: Annotated[str, Form()],
    tags: Annotated[str, Form()] = "",
    featured_image: Annotated[UploadFile | None, File()] = None,
    attachments: Annotated[list[UploadFile], File()] = [],
):
    """
    Submit a blog post with optional images and attachments.
    """
    post = {
        "title": title,
        "content": content,
        "author": author,
        "tags": tags.split(",") if tags else [],
    }
    
    # Handle featured image
    if featured_image:
        image_content = await featured_image.read()
        post["featured_image"] = {
            "filename": featured_image.filename,
            "content_type": featured_image.content_type,
            "size": len(image_content),
        }
    
    # Handle attachments
    post["attachments"] = []
    for attachment in attachments:
        content = await attachment.read()
        post["attachments"].append({
            "filename": attachment.filename,
            "content_type": attachment.content_type,
            "size": len(content),
        })
    
    return {"post": post}
```

#### File Upload Validation

```python
import os
from pathlib import Path
from fastapi import FastAPI, UploadFile, File, HTTPException
from typing import Annotated

app = FastAPI()

# Configuration
ALLOWED_EXTENSIONS = {".jpg", ".jpeg", ".png", ".gif", ".pdf", ".txt", ".doc", ".docx"}
ALLOWED_CONTENT_TYPES = {
    "image/jpeg",
    "image/png",
    "image/gif",
    "application/pdf",
    "text/plain",
    "application/msword",
    "application/vnd.openxmlformats-officedocument.wordprocessingml.document",
}
MAX_FILE_SIZE = 10 * 1024 * 1024  # 10MB


def validate_file(file: UploadFile) -> None:
    """Validate file extension and content type."""
    # Check extension
    if file.filename:
        ext = Path(file.filename).suffix.lower()
        if ext not in ALLOWED_EXTENSIONS:
            raise HTTPException(
                status_code=400,
                detail=f"File extension '{ext}' not allowed. Allowed: {ALLOWED_EXTENSIONS}",
            )
    
    # Check content type
    if file.content_type not in ALLOWED_CONTENT_TYPES:
        raise HTTPException(
            status_code=400,
            detail=f"Content type '{file.content_type}' not allowed.",
        )


@app.post("/upload-validated")
async def upload_validated(
    file: Annotated[UploadFile, File(description="File to upload with validation")],
):
    """
    Upload a file with comprehensive validation.
    """
    # Validate file type
    validate_file(file)
    
    # Read and validate size
    contents = await file.read()
    
    if len(contents) > MAX_FILE_SIZE:
        raise HTTPException(
            status_code=413,  # Payload Too Large
            detail=f"File too large. Maximum size is {MAX_FILE_SIZE // (1024*1024)}MB",
        )
    
    return {
        "filename": file.filename,
        "content_type": file.content_type,
        "size": len(contents),
        "message": "File uploaded successfully",
    }
```

#### Saving Uploaded Files

```python
import os
import uuid
from pathlib import Path
from fastapi import FastAPI, UploadFile, File, HTTPException
from typing import Annotated

app = FastAPI()

# Upload directory
UPLOAD_DIR = Path("uploads")
UPLOAD_DIR.mkdir(exist_ok=True)


def save_upload_file(file: UploadFile, directory: Path) -> Path:
    """
    Save an uploaded file with a unique filename.
    Returns the path to the saved file.
    """
    # Generate unique filename
    file_id = uuid.uuid4().hex[:8]
    original_name = file.filename or "upload"
    
    # Sanitize filename
    safe_name = "".join(c for c in original_name if c.isalnum() or c in "._-")
    unique_name = f"{file_id}_{safe_name}"
    
    file_path = directory / unique_name
    
    # Write file
    with open(file_path, "wb") as f:
        contents = file.file.read()
        f.write(contents)
    
    return file_path


@app.post("/upload-save")
async def upload_and_save(
    file: Annotated[UploadFile, File(description="File to save")],
):
    """
    Upload and save a file to disk.
    """
    try:
        file_path = save_upload_file(file, UPLOAD_DIR)
        return {
            "filename": file.filename,
            "saved_as": file_path.name,
            "path": str(file_path),
            "message": "File saved successfully",
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Failed to save file: {str(e)}")


@app.post("/upload-stream-save")
async def upload_stream_save(file: UploadFile):
    """
    Upload and save a large file using streaming.
    More memory-efficient for large files.
    """
    file_id = uuid.uuid4().hex[:8]
    safe_name = "".join(c for c in (file.filename or "upload") if c.isalnum() or c in "._-")
    file_path = UPLOAD_DIR / f"{file_id}_{safe_name}"
    
    # Stream chunks to disk
    chunk_size = 1024 * 1024  # 1MB chunks
    
    with open(file_path, "wb") as f:
        while chunk := await file.read(chunk_size):
            f.write(chunk)
    
    file_size = file_path.stat().st_size
    
    return {
        "filename": file.filename,
        "saved_as": file_path.name,
        "size": file_size,
        "message": "File saved successfully",
    }
```

#### Complete File Upload Example with Metadata

```python
import os
import uuid
from datetime import datetime
from pathlib import Path
from typing import Annotated

from fastapi import FastAPI, UploadFile, File, Form, HTTPException
from pydantic import BaseModel

app = FastAPI()

UPLOAD_DIR = Path("uploads")
UPLOAD_DIR.mkdir(exist_ok=True)

# Allowed file types
ALLOWED_TYPES = {
    "image/jpeg": "images",
    "image/png": "images",
    "image/gif": "images",
    "application/pdf": "documents",
    "text/plain": "documents",
}
MAX_SIZE = 20 * 1024 * 1024  # 20MB


class FileMetadata(BaseModel):
    id: str
    original_filename: str
    saved_filename: str
    content_type: str
    size: int
    category: str
    description: str | None
    uploaded_at: datetime
    path: str


@app.post("/upload-with-metadata", response_model=FileMetadata)
async def upload_with_metadata(
    file: Annotated[UploadFile, File(description="File to upload")],
    description: Annotated[str | None, Form(max_length=500)] = None,
    category: Annotated[str | None, Form()] = None,
):
    """
    Upload a file with metadata.
    Files are organized into subdirectories by content type.
    """
    # Validate content type
    if file.content_type not in ALLOWED_TYPES:
        raise HTTPException(
            status_code=400,
            detail=f"Content type '{file.content_type}' not allowed. "
            f"Allowed types: {list(ALLOWED_TYPES.keys())}",
        )
    
    # Determine category directory
    file_category = category or ALLOWED_TYPES[file.content_type]
    category_dir = UPLOAD_DIR / file_category
    category_dir.mkdir(exist_ok=True)
    
    # Generate unique filename
    file_id = uuid.uuid4().hex
    ext = Path(file.filename or "file").suffix
    saved_filename = f"{file_id}{ext}"
    file_path = category_dir / saved_filename
    
    # Read and validate size
    contents = await file.read()
    
    if len(contents) > MAX_SIZE:
        raise HTTPException(
            status_code=413,
            detail=f"File too large. Maximum size is {MAX_SIZE // (1024*1024)}MB",
        )
    
    # Save file
    with open(file_path, "wb") as f:
        f.write(contents)
    
    # Create metadata
    metadata = FileMetadata(
        id=file_id,
        original_filename=file.filename or "unknown",
        saved_filename=saved_filename,
        content_type=file.content_type or "application/octet-stream",
        size=len(contents),
        category=file_category,
        description=description,
        uploaded_at=datetime.utcnow(),
        path=str(file_path),
    )
    
    return metadata


@app.get("/files/{category}/{filename}")
async def get_file(category: str, filename: str):
    """
    Serve an uploaded file.
    In production, use a proper static file server or CDN.
    """
    from fastapi.responses import FileResponse
    
    file_path = UPLOAD_DIR / category / filename
    
    if not file_path.exists():
        raise HTTPException(status_code=404, detail="File not found")
    
    return FileResponse(
        path=file_path,
        filename=filename,
        media_type="application/octet-stream",
    )
```

---

### 7.3 Cookies: Reading and Setting Cookies Securely

Cookies are small pieces of data stored in the browser. FastAPI makes it easy to read cookies from requests and set cookies in responses.

#### Reading Cookies

```python
from fastapi import FastAPI, Cookie, Request
from typing import Annotated

app = FastAPI()


@app.get("/read-cookies")
async def read_cookies(request: Request):
    """
    Read all cookies using the Request object.
    """
    return {"cookies": request.cookies}


@app.get("/read-specific-cookie")
async def read_specific_cookie(
    session_id: Annotated[str | None, Cookie()] = None,
    user_preference: Annotated[str | None, Cookie()] = None,
):
    """
    Read specific cookies using the Cookie parameter.
    """
    return {
        "session_id": session_id,
        "user_preference": user_preference,
    }
```

#### Cookie with Validation

```python
from fastapi import FastAPI, Cookie, HTTPException
from typing import Annotated

app = FastAPI()


@app.get("/protected")
async def protected_route(
    auth_token: Annotated[str, Cookie(description="Authentication token")],
):
    """
    Route that requires a valid auth cookie.
    """
    # Validate the token (simplified example)
    if not auth_token.startswith("valid-"):
        raise HTTPException(status_code=401, detail="Invalid auth token")
    
    return {"message": "Access granted", "token": auth_token[:10] + "..."}


@app.get("/preferences")
async def get_preferences(
    theme: Annotated[str, Cookie()] = "light",
    language: Annotated[str, Cookie()] = "en",
    font_size: Annotated[int, Cookie()] = 14,
):
    """
    Get user preferences from cookies with defaults.
    """
    return {
        "theme": theme,
        "language": language,
        "font_size": font_size,
    }
```

#### Setting Cookies

Use the `Response` object to set cookies:

```python
from fastapi import FastAPI, Response, HTTPException
from pydantic import BaseModel

app = FastAPI()


class LoginRequest(BaseModel):
    username: str
    password: str


@app.post("/login")
async def login(response: Response, login_data: LoginRequest):
    """
    Login and set authentication cookies.
    """
    # Simulate authentication
    if login_data.username != "admin" or login_data.password != "secret":
        raise HTTPException(status_code=401, detail="Invalid credentials")
    
    # Set cookies
    response.set_cookie(
        key="auth_token",
        value="valid-token-abc123",
        httponly=True,  # Not accessible via JavaScript
        secure=True,    # Only sent over HTTPS
        samesite="lax", # CSRF protection
        max_age=3600,   # 1 hour
    )
    
    response.set_cookie(
        key="username",
        value=login_data.username,
        httponly=False,  # Accessible via JavaScript if needed
        max_age=3600,
    )
    
    return {"message": "Login successful", "username": login_data.username}


@app.post("/logout")
async def logout(response: Response):
    """
    Logout by clearing cookies.
    """
    response.delete_cookie(key="auth_token")
    response.delete_cookie(key="username")
    
    return {"message": "Logged out successfully"}
```

#### Cookie Parameters

The `set_cookie` method accepts several parameters for security and control:

```python
from fastapi import FastAPI, Response

app = FastAPI()


@app.post("/set-preferences")
async def set_preferences(response: Response):
    """
    Set user preference cookies with various options.
    """
    # Basic cookie
    response.set_cookie(
        key="theme",
        value="dark",
    )
    
    # Secure cookie with all options
    response.set_cookie(
        key="session_id",
        value="abc123xyz",
        max_age=86400,           # Lifetime in seconds (24 hours)
        expires=None,            # Or specific datetime
        path="/",                # URL path for cookie
        domain=None,             # Domain (e.g., ".example.com")
        secure=True,             # Only sent over HTTPS
        httponly=True,           # Not accessible via JavaScript
        samesite="lax",          # "lax", "strict", or "none"
    )
    
    return {"message": "Preferences set"}
```

**Cookie Security Options:**

| Option | Description | Recommendation |
|--------|-------------|----------------|
| `httponly` | Prevents JavaScript access | **Always `True`** for auth tokens |
| `secure` | Only sent over HTTPS | **Always `True`** in production |
| `samesite` | Controls cross-site requests | `"lax"` or `"strict"` for security |
| `max_age` | Lifetime in seconds | Use for session cookies |
| `expires` | Expiration datetime | Alternative to `max_age` |
| `path` | URL path scope | Usually `"/"` |
| `domain` | Domain scope | Only set for cross-subdomain |

#### Complete Cookie-Based Session Example

```python
import os
import hashlib
import hmac
import time
from datetime import datetime, timedelta
from typing import Annotated

from fastapi import FastAPI, Response, Cookie, HTTPException, Depends
from pydantic import BaseModel

app = FastAPI()

# Secret key for signing cookies (use environment variable in production)
SECRET_KEY = os.environ.get("SECRET_KEY", "your-secret-key-change-in-production")
SESSION_EXPIRY = 3600  # 1 hour


class SessionData(BaseModel):
    user_id: int
    username: str
    created_at: float


def create_session_token(user_id: int, username: str) -> str:
    """
    Create a signed session token.
    In production, use proper JWT or session store.
    """
    timestamp = time.time()
    data = f"{user_id}:{username}:{timestamp}"
    signature = hmac.new(
        SECRET_KEY.encode(),
        data.encode(),
        hashlib.sha256,
    ).hexdigest()
    return f"{data}:{signature}"


def verify_session_token(token: str) -> SessionData | None:
    """
    Verify and decode a session token.
    """
    try:
        parts = token.split(":")
        if len(parts) != 4:
            return None
        
        user_id_str, username, timestamp_str, signature = parts
        data = f"{user_id_str}:{username}:{timestamp_str}"
        
        # Verify signature
        expected_signature = hmac.new(
            SECRET_KEY.encode(),
            data.encode(),
            hashlib.sha256,
        ).hexdigest()
        
        if not hmac.compare_digest(signature, expected_signature):
            return None
        
        # Check expiry
        timestamp = float(timestamp_str)
        if time.time() - timestamp > SESSION_EXPIRY:
            return None
        
        return SessionData(
            user_id=int(user_id_str),
            username=username,
            created_at=timestamp,
        )
    except (ValueError, TypeError):
        return None


# Simulated user database
USERS_DB = {
    "admin": {"id": 1, "username": "admin", "password": "admin123"},
    "alice": {"id": 2, "username": "alice", "password": "alice123"},
}


class LoginRequest(BaseModel):
    username: str
    password: str


@app.post("/login")
async def login(response: Response, credentials: LoginRequest):
    """
    Authenticate user and set session cookie.
    """
    user = USERS_DB.get(credentials.username)
    
    if not user or user["password"] != credentials.password:
        raise HTTPException(status_code=401, detail="Invalid credentials")
    
    # Create session token
    token = create_session_token(user["id"], user["username"])
    
    # Set secure cookie
    response.set_cookie(
        key="session",
        value=token,
        httponly=True,
        secure=True,  # Set to True in production with HTTPS
        samesite="lax",
        max_age=SESSION_EXPIRY,
    )
    
    return {
        "message": "Login successful",
        "user": {"id": user["id"], "username": user["username"]},
    }


@app.post("/logout")
async def logout(response: Response):
    """
    Clear session cookie.
    """
    response.delete_cookie(key="session")
    return {"message": "Logged out"}


def get_current_user(
    session: Annotated[str | None, Cookie()] = None,
) -> SessionData:
    """
    Dependency to get the current authenticated user.
    """
    if not session:
        raise HTTPException(status_code=401, detail="Not authenticated")
    
    session_data = verify_session_token(session)
    if not session_data:
        raise HTTPException(status_code=401, detail="Invalid or expired session")
    
    return session_data


UserDep = Annotated[SessionData, Depends(get_current_user)]


@app.get("/me")
async def get_current_user_info(user: UserDep):
    """
    Get current user information.
    """
    return {
        "user_id": user.user_id,
        "username": user.username,
        "session_created": datetime.fromtimestamp(user.created_at).isoformat(),
    }


@app.get("/protected")
async def protected_route(user: UserDep):
    """
    Route that requires authentication.
    """
    return {
        "message": f"Hello, {user.username}!",
        "user_id": user.user_id,
    }
```

---

### 7.4 Headers: Accessing and Manipulating HTTP Headers

Headers carry metadata about requests and responses. FastAPI provides tools for both reading request headers and setting response headers.

#### Reading Request Headers

```python
from fastapi import FastAPI, Header, Request
from typing import Annotated

app = FastAPI()


@app.get("/headers-info")
async def headers_info(request: Request):
    """
    Get all request headers using the Request object.
    """
    return dict(request.headers)


@app.get("/specific-headers")
async def specific_headers(
    user_agent: Annotated[str | None, Header()] = None,
    accept: Annotated[str | None, Header()] = None,
    content_type: Annotated[str | None, Header()] = None,
    authorization: Annotated[str | None, Header()] = None,
):
    """
    Read specific headers using the Header parameter.
    Note: Header names are converted to lowercase with underscores.
    """
    return {
        "user_agent": user_agent,
        "accept": accept,
        "content_type": content_type,
        "authorization": authorization,
    }
```

#### Header with Validation

```python
from fastapi import FastAPI, Header, HTTPException
from typing import Annotated

app = FastAPI()


@app.get("/api-version")
async def api_version(
    x_api_version: Annotated[str, Header(description="API version header")],
):
    """
    Require a specific API version header.
    """
    supported_versions = ["v1", "v2", "v3"]
    
    if x_api_version not in supported_versions:
        raise HTTPException(
            status_code=400,
            detail=f"Unsupported API version. Supported: {supported_versions}",
        )
    
    return {"api_version": x_api_version}


@app.get("/rate-limit")
async def rate_limit(
    x_rate_limit: Annotated[int, Header(ge=1, le=100)] = 10,
):
    """
    Header with validation constraints.
    """
    return {"rate_limit": x_rate_limit}


@app.get("/client-info")
async def client_info(
    x_forwarded_for: Annotated[str | None, Header()] = None,
    x_real_ip: Annotated[str | None, Header()] = None,
    user_agent: Annotated[str | None, Header()] = None,
):
    """
    Get client information from headers.
    Common for reverse proxy setups.
    """
    return {
        "client_ip": x_forwarded_for or x_real_ip or "unknown",
        "user_agent": user_agent,
    }
```

#### Setting Response Headers

Use the `Response` object to set response headers:

```python
from fastapi import FastAPI, Response
from datetime import datetime

app = FastAPI()


@app.get("/custom-headers")
async def custom_headers(response: Response):
    """
    Set custom response headers.
    """
    response.headers["X-Custom-Header"] = "Custom Value"
    response.headers["X-Timestamp"] = datetime.utcnow().isoformat()
    response.headers["X-RateLimit-Limit"] = "100"
    response.headers["X-RateLimit-Remaining"] = "99"
    
    return {"message": "Check the response headers"}


@app.get("/cache-control")
async def cache_control(response: Response):
    """
    Set caching headers.
    """
    response.headers["Cache-Control"] = "public, max-age=3600"
    response.headers["ETag"] = "abc123"
    response.headers["Last-Modified"] = "Mon, 15 Jan 2024 12:00:00 GMT"
    
    return {"data": "This response is cacheable"}
```

#### Using `Response` Classes

FastAPI provides specialized response classes for different content types:

```python
from fastapi import FastAPI
from fastapi.responses import (
    JSONResponse,
    HTMLResponse,
    PlainTextResponse,
    RedirectResponse,
    StreamingResponse,
    FileResponse,
)
import json

app = FastAPI()


@app.get("/json-response")
async def json_response():
    """
    Return a JSONResponse with custom headers.
    """
    return JSONResponse(
        content={"message": "Hello", "status": "success"},
        headers={
            "X-Custom-Header": "JSON Response",
            "X-Cache": "no-cache",
        },
    )


@app.get("/html-response")
async def html_response():
    """
    Return HTML content.
    """
    html_content = """
    <!DOCTYPE html>
    <html>
        <head><title>HTML Response</title></head>
        <body><h1>Hello from FastAPI!</h1></body>
    </html>
    """
    return HTMLResponse(
        content=html_content,
        headers={"Content-Type": "text/html; charset=utf-8"},
    )


@app.get("/text-response")
async def text_response():
    """
    Return plain text.
    """
    return PlainTextResponse(
        content="This is plain text content.",
        headers={"X-Content-Type": "text/plain"},
    )


@app.get("/redirect")
async def redirect():
    """
    Redirect to another URL.
    """
    return RedirectResponse(
        url="/json-response",
        status_code=302,  # or 301 for permanent redirect
    )
```

#### Streaming Responses

For large files or real-time data:

```python
from fastapi import FastAPI
from fastapi.responses import StreamingResponse
import io
import csv

app = FastAPI()


@app.get("/stream-data")
async def stream_data():
    """
    Stream data to the client.
    """
    
    async def generate_data():
        """Generator that yields data chunks."""
        for i in range(10):
            yield f"data: Item {i}\n\n"
    
    return StreamingResponse(
        generate_data(),
        media_type="text/plain",
        headers={
            "X-Stream": "true",
            "Cache-Control": "no-cache",
        },
    )


@app.get("/download-csv")
async def download_csv():
    """
    Generate and stream a CSV file.
    """
    
    def generate_csv():
        """Generate CSV content."""
        output = io.StringIO()
        writer = csv.writer(output)
        
        # Header row
        writer.writerow(["ID", "Name", "Email"])
        yield output.getvalue()
        output.seek(0)
        output.truncate(0)
        
        # Data rows
        for i in range(1, 101):
            writer.writerow([i, f"User {i}", f"user{i}@example.com"])
            yield output.getvalue()
            output.seek(0)
            output.truncate(0)
    
    return StreamingResponse(
        generate_csv(),
        media_type="text/csv",
        headers={
            "Content-Disposition": "attachment; filename=users.csv",
        },
    )
```

#### File Responses

Serve files directly:

```python
from pathlib import Path
from fastapi import FastAPI, HTTPException
from fastapi.responses import FileResponse

app = FastAPI()

FILES_DIR = Path("files")


@app.get("/download/{filename}")
async def download_file(filename: str):
    """
    Download a file from the server.
    """
    file_path = FILES_DIR / filename
    
    if not file_path.exists():
        raise HTTPException(status_code=404, detail="File not found")
    
    return FileResponse(
        path=file_path,
        filename=filename,
        media_type="application/octet-stream",
    )


@app.get("/image/{filename}")
async def serve_image(filename: str):
    """
    Serve an image file with proper content type.
    """
    file_path = FILES_DIR / filename
    
    if not file_path.exists():
        raise HTTPException(status_code=404, detail="Image not found")
    
    # Determine content type based on extension
    content_types = {
        ".jpg": "image/jpeg",
        ".jpeg": "image/jpeg",
        ".png": "image/png",
        ".gif": "image/gif",
        ".webp": "image/webp",
    }
    
    ext = file_path.suffix.lower()
    media_type = content_types.get(ext, "application/octet-stream")
    
    return FileResponse(
        path=file_path,
        media_type=media_type,
    )
```

#### CORS Headers

For cross-origin requests, use FastAPI's CORS middleware:

```python
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

# Configure CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["https://example.com", "https://app.example.com"],
    allow_credentials=True,
    allow_methods=["GET", "POST", "PUT", "DELETE", "OPTIONS"],
    allow_headers=["*"],
    expose_headers=["X-Custom-Header"],
    max_age=3600,
)


@app.get("/cors-test")
async def cors_test():
    """
    This endpoint will have CORS headers.
    """
    return {"message": "CORS is configured"}
```

#### Complete API with Custom Headers

```python
from datetime import datetime
from typing import Annotated
from fastapi import FastAPI, Response, Header, HTTPException, Request
from fastapi.responses import JSONResponse
from pydantic import BaseModel

app = FastAPI(
    title="API with Custom Headers",
    description="Demonstrates working with HTTP headers",
)

# Simulated API key store
API_KEYS = {
    "key-abc123": {"name": "Client App", "rate_limit": 100},
    "key-xyz789": {"name": "Admin App", "rate_limit": 1000},
}


class APIResponse(BaseModel):
    data: dict
    timestamp: str


def verify_api_key(x_api_key: Annotated[str, Header()]):
    """Dependency to verify API key."""
    if x_api_key not in API_KEYS:
        raise HTTPException(
            status_code=401,
            detail="Invalid API key",
            headers={"WWW-Authenticate": "ApiKey"},
        )
    return API_KEYS[x_api_key]


@app.get("/api/data")
async def get_data(
    response: Response,
    api_key: dict = Header(),
):
    """
    Endpoint with custom headers and API key verification.
    """
    # Add custom headers
    response.headers["X-API-Version"] = "1.0"
    response.headers["X-Request-Timestamp"] = datetime.utcnow().isoformat()
    response.headers["X-RateLimit-Limit"] = str(api_key["rate_limit"])
    response.headers["X-RateLimit-Remaining"] = str(api_key["rate_limit"] - 1)
    
    return APIResponse(
        data={"items": [1, 2, 3], "count": 3},
        timestamp=datetime.utcnow().isoformat(),
    )


@app.exception_handler(HTTPException)
async def custom_http_exception_handler(request: Request, exc: HTTPException):
    """
    Custom exception handler that adds headers to error responses.
    """
    return JSONResponse(
        status_code=exc.status_code,
        content={"error": exc.detail, "status_code": exc.status_code},
        headers={
            "X-Error": "true",
            "X-Error-Code": str(exc.status_code),
            "X-Timestamp": datetime.utcnow().isoformat(),
        },
    )
```

---

### Summary

In this chapter, you've learned comprehensive request and response handling:

1. **The `Request` Object**: Direct access to HTTP request details including method, URL, headers, cookies, client info, and body.

2. **Form Data and File Uploads**: Handling `application/x-www-form-urlencoded` and `multipart/form-data` with validation and file saving.

3. **Cookies**: Reading cookies from requests and setting secure cookies in responses with proper security options.

4. **Headers**: Reading request headers, setting response headers, and using specialized response classes for different content types.

---

### Exercises

1. **Request Logger Middleware**: Create a dependency that logs request details including method, path, client IP, user agent, and response time.

2. **File Upload API**: Build an API that:
   - Accepts file uploads with metadata (title, description, tags)
   - Validates file type and size
   - Saves files to organized directories by date
   - Returns a file ID for later retrieval

3. **Cookie-Based Sessions**: Implement a complete session system that:
   - Sets secure session cookies on login
   - Validates sessions on protected routes
   - Implements session refresh
   - Handles logout properly

4. **Custom Response Headers**: Create an API that returns:
   - Rate limit headers
   - Cache control headers
   - Custom pagination headers
   - Request timing headers

---

### What's Next?

**Chapter 8: Response Handling** will explore advanced response features:
- Setting explicit status codes for different scenarios
- Optimizing response performance with `ORJSONResponse`
- Streaming responses for large data
- Custom response classes for HTML, XML, and other formats

